In [1]:
import os


from dotenv import load_dotenv
load_dotenv()

True

In [2]:
cmnds = []

In [3]:
from huggingface_hub import list_models
import pandas as pd

username = "loris3"

# Fetch all models for a specific user
models = list_models(author=username)

df = pd.DataFrame(models)
df = df[["id"]]

In [4]:
pretraining_todo = [
    # 'loris3/stratified_equitoken_10m_curriculum_llama_random',
    # 'loris3/babylm_2024_10m_curriculum_llama_random'
    
]
# df = pd.concat([df, pd.DataFrame( pretraining_todo, columns= ["id"], )])

In [5]:
import re
def extract_type(text):
   text = text.replace("_llama","")
   return text.split("_curriculum_")[-1]

df['type'] = df['id'].apply(extract_type)

# Add 'model_type' column based on the presence of 'llama' or 'bert'
def extract_model_type(text):
    return 'llama' if 'llama' in text else 'bert'

df['model_type'] = df['id'].apply(extract_model_type)

# Add 'dataset_type' column based on specific keywords
def extract_dataset_type(text):
    if 'stratified' in text and not 'equitoken' in text:
        return 'equidocument'
    elif 'babylm' in text:
        return 'babylm'
    elif 'equitoken' in text:
        return 'equitoken'
    return 'unknown'

df['dataset_type'] = df['id'].apply(extract_dataset_type)
pd.set_option('display.max_colwidth', None)
df = df.set_index("id")
df

,type,model_type,dataset_type
id,,,
loris3/stratified_10m_curriculum_random,random,bert,equidocument
loris3/stratified_10m_curriculum_curriculum,curriculum,bert,equidocument
loris3/stratified_10m_curriculum_lognorm,lognorm,bert,equidocument
loris3/stratified_10m_curriculum_lognorm_top_50_pct,lognorm_top_50_pct,bert,equidocument
loris3/stratified_10m_curriculum_lognorm_top_50_pct_shuffled,lognorm_top_50_pct_shuffled,bert,equidocument
loris3/stratified_10m_curriculum_dirac_positive_only,dirac_positive_only,bert,equidocument
loris3/stratified_10m_curriculum_dirac,dirac,bert,equidocument
loris3/stratified_10m_curriculum_lognorm_top_50_pct_twice_shuffled,lognorm_top_50_pct_twice_shuffled,bert,equidocument
loris3/stratified_10m_curriculum_influential_examples_early,influential_examples_early,bert,equidocument


In [6]:
df_pretraining = df[df["type"] == "random"]

display(df_pretraining)

assert len(df_pretraining[df_pretraining["model_type"] == "bert"]) == 3
assert len(df_pretraining[df_pretraining["model_type"] == "llama"]) == 3

,type,model_type,dataset_type
id,,,
loris3/stratified_10m_curriculum_random,random,bert,equidocument
loris3/babylm_2024_10m_curriculum_random,random,bert,babylm
loris3/stratified_equitoken_10m_curriculum_random,random,bert,equitoken
loris3/stratified_10m_curriculum_llama_random,random,llama,equidocument
loris3/stratified_equitoken_10m_curriculum_llama_random,random,llama,equitoken
loris3/babylm_2024_10m_curriculum_llama_random,random,llama,babylm


In [7]:
for _,row in df_pretraining.iterrows():
    name = row.name
    basename = os.path.basename(name)
    influence_path = os.path.join("./mean_influence", basename, "_".join([basename.replace("_random","") + "train"]*2))

    dataset_name = name.replace("_random","").replace("_llama","")
    if not os.path.exists(influence_path) or len(os.listdir(influence_path)) < 10:
        cmnds.append(" ".join(["python3 slurm.py",name, dataset_name, "train",dataset_name, "train","--n_checkpoints=10", "--paradigm="+("mlm" if "llama" not in name else "pre")]))
        print(cmnds[-1])

python3 slurm.py loris3/stratified_10m_curriculum_random loris3/stratified_10m_curriculum train loris3/stratified_10m_curriculum train --n_checkpoints=10 --paradigm=mlm
python3 slurm.py loris3/babylm_2024_10m_curriculum_random loris3/babylm_2024_10m_curriculum train loris3/babylm_2024_10m_curriculum train --n_checkpoints=10 --paradigm=mlm
python3 slurm.py loris3/stratified_equitoken_10m_curriculum_random loris3/stratified_equitoken_10m_curriculum train loris3/stratified_equitoken_10m_curriculum train --n_checkpoints=10 --paradigm=mlm
python3 slurm.py loris3/stratified_10m_curriculum_llama_random loris3/stratified_10m_curriculum train loris3/stratified_10m_curriculum train --n_checkpoints=10 --paradigm=pre
python3 slurm.py loris3/stratified_equitoken_10m_curriculum_llama_random loris3/stratified_equitoken_10m_curriculum train loris3/stratified_equitoken_10m_curriculum train --n_checkpoints=10 --paradigm=pre
python3 slurm.py loris3/babylm_2024_10m_curriculum_llama_random loris3/babylm_20

In [8]:
curricula = ["dirac", "lognorm", "influential_examples_first_half", "influential_examples_sandwich"]
for _,row in df_pretraining.iterrows():
    name = row.name
    for curriculum in curricula:
        if not (name.replace("_random", "_") + curriculum) in df.index:
            cmnds.append(" ".join(["./slurm_pretraining.sh",name.replace("_random", "").replace("_llama", ""), curriculum+".pt", "llama" if "llama" in name else "roberta"]))
            print(cmnds[-1])
    


./slurm_pretraining.sh loris3/stratified_10m_curriculum influential_examples_first_half.pt roberta
./slurm_pretraining.sh loris3/stratified_10m_curriculum influential_examples_sandwich.pt roberta
./slurm_pretraining.sh loris3/babylm_2024_10m_curriculum dirac.pt roberta
./slurm_pretraining.sh loris3/babylm_2024_10m_curriculum lognorm.pt roberta
./slurm_pretraining.sh loris3/babylm_2024_10m_curriculum influential_examples_first_half.pt roberta
./slurm_pretraining.sh loris3/babylm_2024_10m_curriculum influential_examples_sandwich.pt roberta
./slurm_pretraining.sh loris3/stratified_equitoken_10m_curriculum influential_examples_first_half.pt roberta
./slurm_pretraining.sh loris3/stratified_equitoken_10m_curriculum influential_examples_sandwich.pt roberta
./slurm_pretraining.sh loris3/stratified_10m_curriculum dirac.pt llama
./slurm_pretraining.sh loris3/stratified_10m_curriculum lognorm.pt llama
./slurm_pretraining.sh loris3/stratified_10m_curriculum influential_examples_first_half.pt llama

In [9]:
print("\n".join(cmnds))

python3 slurm.py loris3/stratified_10m_curriculum_random loris3/stratified_10m_curriculum train loris3/stratified_10m_curriculum train --n_checkpoints=10 --paradigm=mlm
python3 slurm.py loris3/babylm_2024_10m_curriculum_random loris3/babylm_2024_10m_curriculum train loris3/babylm_2024_10m_curriculum train --n_checkpoints=10 --paradigm=mlm
python3 slurm.py loris3/stratified_equitoken_10m_curriculum_random loris3/stratified_equitoken_10m_curriculum train loris3/stratified_equitoken_10m_curriculum train --n_checkpoints=10 --paradigm=mlm
python3 slurm.py loris3/stratified_10m_curriculum_llama_random loris3/stratified_10m_curriculum train loris3/stratified_10m_curriculum train --n_checkpoints=10 --paradigm=pre
python3 slurm.py loris3/stratified_equitoken_10m_curriculum_llama_random loris3/stratified_equitoken_10m_curriculum train loris3/stratified_equitoken_10m_curriculum train --n_checkpoints=10 --paradigm=pre
python3 slurm.py loris3/babylm_2024_10m_curriculum_llama_random loris3/babylm_20

In [10]:
assert len(set(cmnds)) == len(cmnds) 